<a href="https://colab.research.google.com/github/eslamahmed235/Arabic-dialect-classification-ML-DL/blob/main/Data_pre_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-7vpa0W7lOtiOXCT_XxzH-63ed5Zo7jk' -O dialict.csv
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1-8CiXjf4bmuDS5Q7xVZe3s4iQsua2vER' -O output.csv


--2022-03-14 17:59:19--  https://docs.google.com/uc?export=download&id=1-7vpa0W7lOtiOXCT_XxzH-63ed5Zo7jk
Resolving docs.google.com (docs.google.com)... 172.217.0.46, 2607:f8b0:4004:800::200e
Connecting to docs.google.com (docs.google.com)|172.217.0.46|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-1g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i3qpl6el2vafi5gjj4ki7jvd12lvu0v2/1647280725000/15315052740638448697/*/1-7vpa0W7lOtiOXCT_XxzH-63ed5Zo7jk?e=download [following]
--2022-03-14 17:59:20--  https://doc-08-1g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i3qpl6el2vafi5gjj4ki7jvd12lvu0v2/1647280725000/15315052740638448697/*/1-7vpa0W7lOtiOXCT_XxzH-63ed5Zo7jk?e=download
Resolving doc-08-1g-docs.googleusercontent.com (doc-08-1g-docs.googleusercontent.com)... 142.250.81.193, 2607:f8b0:4004:82f::2001
Connecting to doc-08-1g-docs.googleusercontent.com (doc-08-1g-docs.googleusercontent

In [3]:
import pandas as pd
import numpy as np
import requests
import json as js
from time import sleep

from imblearn.pipeline       import Pipeline 
from sklearn.model_selection import cross_validate
from imblearn.over_sampling  import RandomOverSampler
import numpy as np
from sklearn.metrics import precision_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import seaborn as sns

from sklearn.pipeline import Pipeline
from keras.callbacks import TensorBoard
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from imblearn.under_sampling import RandomUnderSampler

from imblearn.under_sampling import TomekLinks

# !pip install gensim spacy nltk
from sklearn.feature_extraction.text import TfidfVectorizer

!pip install pyarabic
import pyarabic
import pyarabic.araby as araby
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from textblob import TextBlob
from __future__ import unicode_literals
import regex as re

from numpy.ma.core import size
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix


from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import ComplementNB, GaussianNB , BernoulliNB 

import pyarabic.araby as araby
from pyarabic.araby import normalize_ligature
from pyarabic.araby import normalize_hamza
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
text_data = pd.read_csv("/content/output.csv",lineterminator='\n')
label_data = pd.read_csv("/content/dialict.csv")


In [5]:
# Drop unimportant column
text_data.drop('Unnamed: 0', axis=1, inplace=True)

# Merge fetched Data and label by tweet ID
Data = pd.merge(text_data, label_data, how='inner', left_on = 'tweet_id', right_on = 'id')
Data.drop('id', axis=1, inplace=True)

Data.head()

,tweet_id,tweet,dialect
0,1175358310087892992,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .,IQ
1,1175416117793349632,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...,IQ
2,1175450108898565888,@KanaanRema مبين من كلامه خليجي,IQ
3,1175471073770573824,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐,IQ
4,1175496913145217024,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺,IQ


In [6]:
# function to clean hashtag and mention from text
def clean_hashtag_and_mention (text):
  
  text = re.sub("@[A-Za-z0-9_]+","", text)
  text = re.sub("#[A-Za-z0-9_]+","", text)
  return text

def clean_tweet(text):
    text = re.sub('#\d+K\d+', ' ', text)  # years like 2K19
    text = re.sub('http\S+\s*', ' ', text)  # remove URLs
    text = re.sub('RT|cc', ' ', text)  # remove RT and cc
    text = re.sub('@[^\s]+',' ',text)
    text = clean_hashtag_and_mention(text)
    return text


In [7]:
# fuction to remove emojis by regex 

def remove_emoji(text):
    emoji_pattern = re.compile("["
                                   u"\U0001F600-\U0001F64F"  # emoticons
                                   u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                                   u"\U0001F680-\U0001F6FF"  # transport & map symbols
                                   u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                                   u"\U00002702-\U000027B0"
                                   u"\U000024C2-\U0001F251"
                                   "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text

# Data['clean_tweet'] = Data['clean_tweet'].apply(lambda x:remove_emoji(x))
# Data

In [9]:
# call and save most stopwords in arabic
stops = set(stopwords.words("arabic"))
stop_word_comp = {"،","آض","آمينَ","آه","آهاً","آي","أ","أب","أجل","أجمع","أخ","أخذ","أصبح","أضحى","أقبل","أقل","أكثر","ألا","أم","أما","أمامك","أمامكَ","أمسى","أمّا","أن","أنا","أنت","أنتم","أنتما","أنتن","أنتِ","أنشأ","أنّى","أو","أوشك","أولئك","أولئكم","أولاء","أولالك","أوّهْ","أي","أيا","أين","أينما","أيّ","أَنَّ","أََيُّ","أُفٍّ","إذ","إذا","إذاً","إذما","إذن","إلى","إليكم","إليكما","إليكنّ","إليكَ","إلَيْكَ","إلّا","إمّا","إن","إنّما","إي","إياك","إياكم","إياكما","إياكن","إيانا","إياه","إياها","إياهم","إياهما","إياهن","إياي","إيهٍ","إِنَّ","ا","ابتدأ","اثر","اجل","احد","اخرى","اخلولق","اذا","اربعة","ارتدّ","استحال","اطار","اعادة","اعلنت","اف","اكثر","اكد","الألاء","الألى","الا","الاخيرة","الان","الاول","الاولى","التى","التي","الثاني","الثانية","الذاتي","الذى","الذي","الذين","السابق","الف","اللائي","اللاتي","اللتان","اللتيا","اللتين","اللذان","اللذين","اللواتي","الماضي","المقبل","الوقت","الى","اليوم","اما","امام","امس","ان","انبرى","انقلب","انه","انها","او","اول","اي","ايار","ايام","ايضا","ب","بات","باسم","بان","بخٍ","برس","بسبب","بسّ","بشكل","بضع","بطآن","بعد","بعض","بك","بكم","بكما","بكن","بل","بلى","بما","بماذا","بمن","بن","بنا","به","بها","بي","بيد","بين","بَسْ","بَلْهَ","بِئْسَ","تانِ","تانِك","تبدّل","تجاه","تحوّل","تلقاء","تلك","تلكم","تلكما","تم","تينك","تَيْنِ","تِه","تِي","ثلاثة","ثم","ثمّ","ثمّة","ثُمَّ","جعل","جلل","جميع","جير","حار","حاشا","حاليا","حاي","حتى","حرى","حسب","حم","حوالى","حول","حيث","حيثما","حين","حيَّ","حَبَّذَا","حَتَّى","حَذارِ","خلا","خلال","دون","دونك","ذا","ذات","ذاك","ذانك","ذانِ","ذلك","ذلكم","ذلكما","ذلكن","ذو","ذوا","ذواتا","ذواتي","ذيت","ذينك","ذَيْنِ","ذِه","ذِي","راح","رجع","رويدك","ريث","رُبَّ","زيارة","سبحان","سرعان","سنة","سنوات","سوف","سوى","سَاءَ","سَاءَمَا","شبه","شخصا","شرع","شَتَّانَ","صار","صباح","صفر","صهٍ","صهْ","ضد","ضمن","طاق","طالما","طفق","طَق","ظلّ","عاد","عام","عاما","عامة","عدا","عدة","عدد","عدم","عسى","عشر","عشرة","علق","على","عليك","عليه","عليها","علًّ","عن","عند","عندما","عوض","عين","عَدَسْ","عَمَّا","غدا","غير","ـ","ف","فان","فلان","فو","فى","في","فيم","فيما","فيه","فيها","قال","قام","قبل","قد","قطّ","قلما","قوة","كأنّما","كأين","كأيّ","كأيّن","كاد","كان","كانت","كذا","كذلك","كرب","كل","كلا","كلاهما","كلتا","كلم","كليكما","كليهما","كلّما","كلَّا","كم","كما","كي","كيت","كيف","كيفما","كَأَنَّ","كِخ","لئن","لا","لات","لاسيما","لدن","لدى","لعمر","لقاء","لك","لكم","لكما","لكن","لكنَّما","لكي","لكيلا","للامم","لم","لما","لمّا","لن","لنا","له","لها","لو","لوكالة","لولا","لوما","لي","لَسْتَ","لَسْتُ","لَسْتُم","لَسْتُمَا","لَسْتُنَّ","لَسْتِ","لَسْنَ","لَعَلَّ","لَكِنَّ","لَيْتَ","لَيْسَ","لَيْسَا","لَيْسَتَا","لَيْسَتْ","لَيْسُوا","لَِسْنَا","ما","ماانفك","مابرح","مادام","ماذا","مازال","مافتئ","مايو","متى","مثل","مذ","مساء","مع","معاذ","مقابل","مكانكم","مكانكما","مكانكنّ","مكانَك","مليار","مليون","مما","ممن","من","منذ","منها","مه","مهما","مَنْ","مِن","نحن","نحو","نعم","نفس","نفسه","نهاية","نَخْ","نِعِمّا","نِعْمَ","ها","هاؤم","هاكَ","هاهنا","هبّ","هذا","هذه","هكذا","هل","هلمَّ","هلّا","هم","هما","هن","هنا","هناك","هنالك","هو","هي","هيا","هيت","هيّا","هَؤلاء","هَاتانِ","هَاتَيْنِ","هَاتِه","هَاتِي","هَجْ","هَذا","هَذانِ","هَذَيْنِ","هَذِه","هَذِي","هَيْهَاتَ","و","و6","وا","واحد","واضاف","واضافت","واكد","وان","واهاً","واوضح","وراءَك","وفي","وقال","وقالت","وقد","وقف","وكان","وكانت","ولا","ولم","ومن","مَن","وهو","وهي","ويكأنّ","وَيْ","وُشْكَانََ","يكون","يمكن","يوم","ّأيّان"}

# remove stop_words function 
def remove_stop_words(text):
    zen = TextBlob(text)
    words = zen.words
    return " ".join([w for w in words if not w in stops and not w in stop_word_comp and len(w) >= 2])

# Data['clean_tweet'] = Data['clean_tweet'].apply(lambda x:remove_stop_words(x))


In [10]:
# apply normalization and remove diacritics (Tashkeel)

def normalizeArabic(text):
    text = text.strip()
    text = re.sub("[إأٱآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    noise = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    text = re.sub(noise, '', text)
    text = re.sub(r'(.)\1+', r"\1\1", text) # Remove longation
    return araby.strip_tashkeel(text)

# Data['clean_tweet'] = Data['clean_tweet'].apply(lambda x:normalizeArabic(x))
# Data

In [11]:
def clean_text(text):
## Clean for tweets
  text = clean_tweet(text)
## Remove Emojis
  text = remove_emoji(text)
## Remove Tashkeel
  text = normalizeArabic(text) 
## Remove punctuations
  text = re.sub('[%s]' % re.escape("""!"#$%&'()*+,،-./:;<=>؟?@[\]^_`{|}~"""), ' ', text)  # remove punctuation
## Remove stop words
  text = remove_stop_words(text)
## remove extra whitespace
  text = re.sub('\s+', ' ', text)  
# ## Remove numbers
  text = re.sub("\d+", " ", text)
  text = re.sub('\W+', ' ', text) 
  text = re.sub('[A-Za-z]+',' ',text)
  text = re.sub(r'\\u[A-Za-z0-9\\]+',' ',text)
#   ## remove extra whitespace
  text = re.sub('\s+', ' ', text)  

  return text

Data['clean_tweet'] = Data['tweet'].apply(lambda x:clean_text(x))
Data

,tweet_id,tweet,dialect,clean_tweet
0,1175358310087892992,@Nw8ieJUwaCAAreT لكن بالنهاية .. ينتفض .. يغير .,IQ,بالنهايه ينتفض يغير
1,1175416117793349632,@7zNqXP0yrODdRjK يعني هذا محسوب على البشر .. ح...,IQ,يعني محسوب علي البشر حيونه ووحشيه وتطلبون الغر...
2,1175450108898565888,@KanaanRema مبين من كلامه خليجي,IQ,مبين كلامه خليجي
3,1175471073770573824,@HAIDER76128900 يسلملي مرورك وروحك الحلوه💐,IQ,يسلملي مرورك وروحك الحلوه
4,1175496913145217024,@hmo2406 وين هل الغيبه اخ محمد 🌸🌺,IQ,وين الغيبه اخ محمد
...,...,...,...,...
458192,1019484980282580992,@Al_mhbaa_7 مبسوطين منك اللي باسطانا😅,BH,مبسوطين منك اللي باسطانا
458193,1021083283709407232,@Zzainabali @P_ameerah والله ماينده ابش يختي,BH,والله ماينده ابش يختي
458194,1017477537889431552,@Al_mhbaa_7 شو عملنا لك حنا تهربي مننا احنا مس...,BH,شو عملنا حنا تهربي مننا احنا مساكين ليش بتعملي...
458195,1022430374696239232,@haneenalmwla الله يبارك فيها وبالعافيه 😋😋😋,BH,الله يبارك وبالعافيه


In [12]:
# split clean words 
def  tokenize_text(df, text_field, new_text_field_name):
     df[new_text_field_name] = df[text_field].apply(lambda x:' +'.join(  araby.tokenize(x)))
    
     return df
dftoken= tokenize_text(Data,"clean_tweet","clean_tweet_split")  

In [13]:
# ### Common word removal
freq = pd.Series(' '.join(Data['clean_tweet_split']).split()).value_counts()[:12]
freq = list(freq.index)
Data['clean_tweet_split'] = Data['clean_tweet_split'].apply(lambda x: " ".join(x for x in x.split() if x not in freq))
print("Done remove the following most Common words : ",freq )

Done remove the following most Common words :  ['+اللي', '+علي', '+الله', '+مش', '+انا', '+شي', '+والله', '+انت', '+الي', '+عشان', '+الناس', 'انا']
